<a href="https://colab.research.google.com/github/catarina-moreira/causabilityXAi/blob/master/Diabetes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Demystifying Predictive Black-Box Models: An Interpretable Probabilistic Approach

Catarina Moreira, Yu-Liang Chou, Mythreyi Velmurugan, Renuka Sindhgatta Rajan, Chun Ouyang, Peter Bruza

**Abstract** 


In [ ]:
# Install tensorflow
try:
    # tensorflow_version only exists in Colab
    %tensorflow_version 2.x
except Exception:
    pass

In [ ]:
# library to deal with Bayesian Networks
#!pip install pyagrum

In [ ]:
import lime
from lime import lime_tabular


In [ ]:
# for reproduciability reasons:
import numpy as np
import pandas as pd
import random as rn
import time

%matplotlib inline

# import auxiliary functions
from learning import *

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)


In [ ]:
# use only if opening on google colab
#from google.colab import drive
#drive.mount('/content/drive')

## Diabetes Dataset

**Context**
This dataset is originally from the National Institute of Diabetes and Digestive and Kidney Diseases. The objective of the dataset is to diagnostically predict whether or not a patient has diabetes, based on certain diagnostic measurements included in the dataset. Several constraints were placed on the selection of these instances from a larger database. In particular, all patients here are females at least 21 years old of Pima Indian heritage.

**Content**
The datasets consists of several medical predictor variables and one target variable, Outcome. Predictor variables includes the 
- number of pregnancies the patient has had, 
- their BMI, 
- insulin level, 
- age,
- glucose,
- blood pressure,
- skin thickness,
- Diabetes pedigree function


### Checking Dataset

In [ ]:
# path to project folder
# please change to your own
PATH = "/Users/catarina/GitHub/causabilityXAi/"

In [ ]:
# name of dataset
DATASET_NAME = "diabetes.csv"

# variable containing the class labels in this case the dataset contains:
# 0 - if not diabetes
# 1 - if diabetes
class_var = "Outcome"

# load dataset
dataset_path = PATH + "datasets/" + DATASET_NAME
data = pd.read_csv( dataset_path )

In [ ]:
# features
feature_names = data.drop([class_var], axis=1).columns.to_list()

# check how balanced the classes are
data.groupby(class_var).count()

### Balanced Dataset

In [ ]:
# balance dataset
sampled_data = data.sample(frac=1)
sampled_data = sampled_data[ sampled_data["Outcome"] == 0]
no_data = sampled_data.sample(frac=1)[0:268]

yes_data = data[ data["Outcome"] == 1]

balanced_data = [no_data,yes_data]
balanced_data = pd.concat(balanced_data)

# check how balanced the classes are
balanced_data.groupby(class_var).count()

#### Train a Model for the Balanced Dataset

In [ ]:
# apply one hot encoder to data
# standardize the input between 0 and 1
X, Y, encoder, scaler = encode_data( balanced_data, class_var)

n_features = X.shape[1]
n_classes = len(balanced_data[class_var].unique())
 
flag = False  # DO NOT CHANGE! Data has already been generated. 
if flag:
    # save training, test and validation data
    generate_save_training_data( dataset_path, X, Y)
    
else:
    # load existing training data
    X_train, Y_train, X_test, Y_test, X_validation, Y_validation= load_training_data( dataset_path )
    

In [ ]:
# generate models for grid search
if flag:
    models = grid_search_model_generator( n_features, n_classes )

    # perform grid_search
    HISTORY_DICT = perform_grid_search( models, PATH, DATASET_NAME.replace(".csv",""), 
                                   X_train, Y_train, 
                                   X_validation, Y_validation, X_test, Y_test, 
                                   batch_size=8, epochs=150 )

In [ ]:
path_serialisation_model = PATH + "training/" + DATASET_NAME.replace(".csv", "") + "/model/" 
path_serialisation_histr = PATH + "training/" + DATASET_NAME.replace(".csv", "") + "/history/" 

# the best performing model was obtained with 5 hidden layers with 12 neurons each
model_name = "model_h5_N12"
    
if flag:
    
    # get respective model training history and model
    model_history = HISTORY_DICT[ model_name ][0]
    model = HISTORY_DICT[ model_name ][1]

    # save model and model history to file
    save_model_history(  model_history, model_name, path_serialisation_histr )
    save_model( model, model_name, path_serialisation_model )
else:
    model_history = load_model_history( model_name, path_serialisation_histr )
    model = load_model( model_name, path_serialisation_model )
    
model.summary()

#### Evaluate Model

In [ ]:
# evaluate loaded model on test and training data
optim = keras.optimizers.Nadam(lr=0.0001, beta_1=0.9, beta_2=0.999)
model.compile(loss='categorical_crossentropy', optimizer=optim, metrics=['accuracy'])

train_loss, train_acc = model.evaluate(X_train, Y_train, verbose=1)
test_loss, test_acc = model.evaluate(X_test, Y_test, verbose=1)

print('\n[Accuracy] Train: %.3f, Test: %.3f' % (train_acc, test_acc))
print('[Loss] Train: %.3f, Test: %.3f' % (train_loss, test_loss))

In [ ]:
#plot_model_history( model_history, 'Accuracy' )
#plot_model_history( model_history, 'Loss' )


In [ ]:
# plot ROC curve
#plot_ROC_Curve( model, X_test, Y_test, n_classes)



### Searching for specific datapoints for local evaluation

In [ ]:

local_data_dict = generate_local_predictions( X_test, Y_test, model, scaler, encoder )

# wrapping up information
true_positives = []
true_negatives = []
false_positives = []
false_negatives = []
for instance in local_data_dict:
    
    if( instance['prediction_type'] == 'TRUE POSITIVE'):
        true_positives.append(instance)

    if( instance['prediction_type'] == 'TRUE NEGATIVE' ):
        true_negatives.append(instance)
        
    if( instance['prediction_type'] == 'FALSE POSITIVE' ):
        false_positives.append(instance)
        
    if( instance['prediction_type'] == 'FALSE NEGATIVE' ):
        false_negatives.append(instance)

### Generating Explanations with Bayesian Networks

#### FALSE POSITIVES

In [ ]:
label_lst = ["No", "Yes"]
class_var = "Diabetes?"

for instance in false_positives:

    # get instance index
    indx = instance['index']
    print("INDEX = " + str(indx))
    
    [bn, inference, infoBN] = generate_BN_explanations(instance, label_lst, feature_names, 
                                                        class_var, encoder, scaler, model, PATH, DATASET_NAME )
 

## Train a Model for the Unbalanced Dataset

In [ ]:
# apply one hot encoder to data
# standardize the input between 0 and 1
X_unb, Y_unb, encoder_unb, scaler = encode_data( data, class_var)

n_features = X_unb.shape[1]
n_classes = len(data[class_var].unique())

flag = False
if flag:
    # save training, test and validation data
    generate_save_training_data( dataset_path + "_unb", X_unb, Y_unb)
    
else:
    # load existing training data
    X_train_unb, Y_train_unb, X_test_unb, Y_test_unb, X_validation_unb, Y_validation_unb= load_training_data( dataset_path + "_unb" )
    